<a href="https://colab.research.google.com/github/nwon24/nlp/blob/main/W4/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN Using PyTorch

For some reason, the cost never goes below around 0.69. :(

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device=torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

data=pd.read_csv("corpus.csv",encoding="latin-1")

tfidf  = TfidfVectorizer()
corpus = tfidf.fit_transform(data["text"])
train_corpus, test_corpus, train_label, test_label = model_selection.train_test_split(data["text"],data['label'],test_size=0.3)
Encoder = LabelEncoder()
train_label = Encoder.fit_transform(train_label)
test_label = Encoder.fit_transform(test_label)
train_corpus_tfidf=tfidf.transform(train_corpus)
test_corpus_tfidf=tfidf.transform(test_corpus)

bsize=20
epochs=10
lr=1e-7

class dense_classifier(nn.Module):
    def __init__(self,n):
        super(dense_classifier,self).__init__()
        self.n=n
        self.network=nn.Sequential(
            nn.Sigmoid(),
            nn.Linear(self.n,10000),
            nn.Tanh(),
            nn.Linear(10000,360),
            #nn.ReLU(),
            nn.Linear(360,1),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.network(x)

class classifier(nn.Module):
    def __init__(self,n,kerns,pool_kerns,out_chans):
        super(classifier,self).__init__()
        self.kerns=kerns
        self.pool_kerns=pool_kerns
        self.convs=[]
        self.pools=[]
        self.stride=1
        self.n=n
        #self.embed=nn.Embedding(n,max_length)
        in_chan=1
        for i in range(len(kerns)):
            out_chan=out_chans[i]
            kern=kerns[i]
            pool_kern=pool_kerns[i]
            self.convs.append(nn.Conv1d(in_chan,out_chan,kern))
            self.compute_size(kern,self.stride)
            self.pools.append(nn.MaxPool1d(pool_kern))
            self.compute_size(pool_kern,pool_kern)
            in_chan=out_chan
        self.convs=nn.ModuleList(self.convs)
        self.pools=nn.ModuleList(self.pools)
        self.fc1=nn.Linear(self.n*out_chan,100)
        self.fc2=nn.Linear(100,1)

    def compute_size(self,kernel,stride):
        self.n=(self.n-kernel)//stride+1
        return self.n

    def forward(self,x):
        #x=self.embed(x)
        for i in range(len(self.kerns)):
            x=nn.ReLU()(self.convs[i](x))
            x=self.pools[i](x)
        x=torch.flatten(x,1)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        #x=nn.Sigmoid()(x)
        return x

class corpus(Dataset):
    def __init__(self,corpus,label):
        self.corpus=corpus
        self.label=label
    def __len__(self):
        return len(self.corpus)
    def __getitem__(self,idx):
        return self.corpus[idx],self.label[idx]

def train(dataloader,model,loss_fn,optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch,(x,y) in enumerate(dataloader):
        x,y=x.to(device),y.to(device)
        optimizer.zero_grad()
        #print(torch.sum(x[0]!=0))
        pred=model(x.reshape(bsize,1,x.shape[1]).to(device))
        cost=loss_fn(pred.flatten(),y)
        cost.backward()
        optimizer.step()
        if batch % 10 == 0:
            cost, current = cost.item(), batch * bsize + len(x)
            print(f"cost: {cost:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for x, y in dataloader:
            x,y=x.to(device),y.to(device)
            pred=model(x.reshape(bsize,1,x.shape[1]).to(device))
            test_loss += loss_fn(pred.flatten(), y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

text="ajbjdfowd"
#input =torch.from_numpy(tfidf.transform([text]).toarray()).float()
train_corpus_data=torch.from_numpy(train_corpus_tfidf.toarray()).float()
test_corpus_data=torch.from_numpy(test_corpus_tfidf.toarray()).float()
train_label_data=torch.from_numpy(train_label).float()
test_label_data=torch.from_numpy(test_label).float()

train_loader=DataLoader(corpus(train_corpus_data,train_label_data),batch_size=bsize)
test_loader=DataLoader(corpus(test_corpus_data,test_label_data),batch_size=bsize)

#input=input.reshape(1,input.shape[0],input.shape[1]
#c=classifier(train_corpus_data.shape[1],[2,3,4,5],[2,2,2,2],[6,16,24,32])
cnn_classifier=classifier(train_corpus_data.shape[1],[10,20],[2,2],[5,10]).to(device)
#input=input.reshape(1,input.shape[0],input.shape[1])
#print(c.forward(input))
dense_classifier=dense_classifier(train_corpus_data.shape[1]).to(device)

c=cnn_classifier

#loss=nn.BCELoss()
loss=nn.BCEWithLogitsLoss()
#optim=torch.optim.SGD(c.parameters(),momentum=0.8,lr=lr)
optim=torch.optim.Adam(c.parameters(),lr=lr)
#optim=torch.optim.ASGD(c.parameters(),lr=lr)

#print(torch.sum(train_label_data==1))

for epoch in range(epochs):
    train(train_loader,c,loss,optim)
test_loop(test_loader,c,loss)

cost: 0.708766  [   20/ 7000]
cost: 0.697480  [  220/ 7000]
cost: 0.686464  [  420/ 7000]
cost: 0.697473  [  620/ 7000]
cost: 0.711969  [  820/ 7000]
cost: 0.693778  [ 1020/ 7000]
cost: 0.697315  [ 1220/ 7000]
cost: 0.683127  [ 1420/ 7000]
cost: 0.683118  [ 1620/ 7000]
cost: 0.683117  [ 1820/ 7000]
cost: 0.690228  [ 2020/ 7000]
cost: 0.675940  [ 2220/ 7000]
cost: 0.690174  [ 2420/ 7000]
cost: 0.711930  [ 2620/ 7000]
cost: 0.690214  [ 2820/ 7000]
cost: 0.700955  [ 3020/ 7000]
cost: 0.700940  [ 3220/ 7000]
cost: 0.693770  [ 3420/ 7000]
cost: 0.686677  [ 3620/ 7000]
cost: 0.700944  [ 3820/ 7000]
cost: 0.704497  [ 4020/ 7000]
cost: 0.693795  [ 4220/ 7000]
cost: 0.700865  [ 4420/ 7000]
cost: 0.711454  [ 4620/ 7000]
cost: 0.700841  [ 4820/ 7000]
cost: 0.690213  [ 5020/ 7000]
cost: 0.690197  [ 5220/ 7000]
cost: 0.683049  [ 5420/ 7000]
cost: 0.700912  [ 5620/ 7000]
cost: 0.700830  [ 5820/ 7000]
cost: 0.690240  [ 6020/ 7000]
cost: 0.697205  [ 6220/ 7000]
cost: 0.690305  [ 6420/ 7000]
cost: 0.68

# Keras

Yay, it works! (A bit of overfitting is happening, but at least the first few epochs look good.)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import InputLayer
from keras.preprocessing import sequence
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection

data=pd.read_csv("corpus.csv",encoding="latin-1")

tfidf  = TfidfVectorizer()
corpus = tfidf.fit_transform(data["text"])
train_corpus, test_corpus, train_label, test_label = model_selection.train_test_split(data["text"],data['label'],test_size=0.3)
Encoder = LabelEncoder()
train_label = Encoder.fit_transform(train_label)
test_label = Encoder.fit_transform(test_label)
train_corpus_tfidf=tfidf.transform(train_corpus)
test_corpus_tfidf=tfidf.transform(test_corpus)

bsize=20
epochs=10
lr=1e-7

features=train_corpus_tfidf.shape[1]
train_corpus_tfidf=train_corpus_tfidf.toarray()
test_corpus_tfidf=test_corpus_tfidf.toarray()
train_corpus_tfidf=train_corpus_tfidf.reshape(train_corpus_tfidf.shape[0],train_corpus_tfidf.shape[1],1)
test_corpus_tfidf=test_corpus_tfidf.reshape(test_corpus_tfidf.shape[0],test_corpus_tfidf.shape[1],1)

model=Sequential([InputLayer(shape=(features,1),batch_size=bsize),
                  Conv1D(64,32,activation="relu"),
                  MaxPooling1D(),
                  Dropout(0.5),
                  Conv1D(32,16,activation="relu"),
                  MaxPooling1D(),
                  Dropout(0.2),
                  Conv1D(18,8,activation="relu"),
                  MaxPooling1D(),
                  Dropout(0.2),
                  Flatten(),
                  Dense(100,activation="relu"),
                  Dense(50,activation="relu"),
                  Dense(1,activation="sigmoid")
                  ])
#model=Sequential([InputLayer(shape=(features,)),
#                  Dense(100,activation="relu"),
#                  Dense(1,activation="sigmoid")
#                  ])
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.summary()

model.fit(train_corpus_tfidf, train_label, epochs=epochs, batch_size=bsize, validation_data=(test_corpus_tfidf, test_label))

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_8 (Conv1D)               │ (20, 31596, 64)        │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_8 (MaxPooling1D)  │ (20, 15798, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (20, 15798, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ (20, 15783, 32)        │        32,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_9 (MaxPooling1D)  │ (20, 7891, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (20, 7891, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_10 (Conv1D)              │ (20, 7884, 18)         │         4,626 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (20, 3942, 18)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (20, 3942, 18)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (20, 70956)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (20, 100)              │     7,095,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (20, 50)               │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (20, 1)                │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,140,339 (27.24 MB)

 Trainable params: 7,140,339 (27.24 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 50s 120ms/step - accuracy: 0.6141 - loss: 0.6222 - val_accuracy: 0.8203 - val_loss: 0.3966
Epoch 2/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 75s 115ms/step - accuracy: 0.8457 - loss: 0.3616 - val_accuracy: 0.8360 - val_loss: 0.3698
Epoch 3/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 41s 115ms/step - accuracy: 0.9094 - loss: 0.2162 - val_accuracy: 0.8423 - val_loss: 0.3727
Epoch 4/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 41s 114ms/step - accuracy: 0.9672 - loss: 0.0913 - val_accuracy: 0.8247 - val_loss: 0.6252
Epoch 5/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 41s 115ms/step - accuracy: 0.9913 - loss: 0.0314 - val_accuracy: 0.8227 - val_loss: 0.8427
Epoch 6/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 41s 114ms/step - accuracy: 0.9973 - loss: 0.0116 - val_accuracy: 0.8273 - val_loss: 0.9651
Epoch 7/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 40s 110ms/step - accuracy: 0.9958 - loss: 0.0120 - val_accuracy: 0.8173 - val_loss: 0.9560
Epoch 8/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 40s 114ms/step - accuracy: 0.9957 - loss: 0